In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import io
data=pd.read_csv('tripcsvbame.csv')

In [6]:
data['usage-stats/'][:2]

0         01aJourneyDataExtract10Jan16-23Jan16.csv
1     01b Journey Data Extract 24Jan16-06Feb16.csv
Name: usage-stats/, dtype: object

In [7]:
dat=data['usage-stats/'][:2]
frames=[]
for x in dat:
    frames.append(pd.read_csv(x[1:]))

result = pd.concat(frames)
result.columns

Index(['Rental Id', 'Duration', 'Bike Id', 'End Date', 'EndStation Id',
       'EndStation Name', 'Start Date', 'StartStation Id',
       'StartStation Name'],
      dtype='object')

In [9]:
date_format = '%d/%m/%Y %H:%M:%S'
result['End Date']=pd.to_datetime(result['End Date'],format='mixed')
result['Start Date']=pd.to_datetime(result['Start Date'],format='mixed')
result

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name
0,50754225,240,11834,2016-01-10 00:04:00,383.0,"Frith Street, Soho",2016-01-10 00:00:00,18,"Drury Lane, Covent Garden"
1,50754226,300,9648,2016-01-10 00:05:00,719.0,"Victoria Park Road, Hackney Central",2016-01-10 00:00:00,479,"Pott Street, Bethnal Green"
2,50754227,1200,10689,2016-01-10 00:20:00,272.0,"Baylis Road, Waterloo",2016-01-10 00:00:00,425,"Harrington Square 2, Camden Town"
3,50754228,780,8593,2016-01-10 00:14:00,471.0,"Hewison Street, Old Ford",2016-01-10 00:01:00,487,"Canton Street, Poplar"
4,50754229,600,8619,2016-01-10 00:11:00,399.0,"Brick Lane Market, Shoreditch",2016-01-10 00:01:00,501,"Cephas Street, Bethnal Green"
...,...,...,...,...,...,...,...,...,...
292694,51360634,600,10400,2016-02-07 00:07:00,48.0,"Godliman Street, St. Paul's",2016-02-06 23:57:00,388,"Southampton Street, Strand"
292695,51360635,240,2320,2016-02-07 00:01:00,574.0,"Eagle Wharf Road, Hoxton",2016-02-06 23:57:00,73,"Old Street Station, St. Luke's"
292696,51360636,720,11143,2016-02-07 00:09:00,535.0,"Gloucester Avenue, Camden Town",2016-02-06 23:57:00,109,"Soho Square , Soho"
292697,51360637,900,6848,2016-02-07 00:14:00,732.0,"Duke Street Hill, London Bridge",2016-02-06 23:59:00,443,"Philpot Street, Whitechapel"


In [11]:
result=result.dropna()
new_result=result.set_index('Start Date').groupby([pd.Grouper(freq='h'), 'StartStation Name', 'EndStation Name']).count()
new_result

Rental Id  \
Start Date          StartStation Name                 EndStation Name                                       
2016-01-10 00:00:00 Abyssinia Close, Clapham Junction West Kensington Station, West Kensington          1   
                    Alderney Street, Pimlico          Chelsea Bridge, Pimlico                           1   
                                                      Grosvenor Crescent, Belgravia                     1   
                    All Saints Church, Portobello     Devonshire Terrace, Bayswater                     1   
                    Ashley Place, Victoria            Ashley Place, Victoria                            4   
...                                                                                                   ...   
2016-02-06 23:00:00 Whiston Road, Haggerston          New Inn Yard, Shoreditch                          1   
                    Winsland Street, Paddington       Warren Street Station, Euston                     1   
                    Woodstock Street, Mayfair         Imperial Wharf Station                            1   
                    York Hall, Bethnal Green          Shoreditch Park, Hoxton                           1   
                                                      Tanner Street, Bermondsey                         1   

                                                                                                Duration  \
Start Date          StartStation Name                 EndStation Name                                      
2016-01-10 00:00:00 Abyssinia Close, Clapham Junction West Kensington Station, West Kensington         1   
                    Alderney Street, Pimlico          Chelsea Bridge, Pimlico                          1   
                                                      Grosvenor Crescent, Belgravia                    1   
                    All Saints Church, Portobello     Devonshire Terrace, Bayswater                    1   
                    Ashley Place, Victoria            Ashley Place, Victoria                           4   
...                                                                                                  ...   
2016-02-06 23:00:00 Whiston Road, Haggerston          New Inn Yard, Shoreditch                         1   
                    Winsland Street, Paddington       Warren Street Station, Euston                    1   
                    Woodstock Street, Mayfair         Imperial Wharf Station                           1   
                    York Hall, Bethnal Green          Shoreditch Park, Hoxton                          1   
                                                      Tanner Street, Bermondsey                        1   

                                                                                                Bike Id  \
Start Date          StartStation Name                 EndStation Name                                     
2016-01-10 00:00:00 Abyssinia Close, Clapham Junction West Kensington Station, West Kensington        1   
                    Alderney Street, Pimlico          Chelsea Bridge, Pimlico                         1   
                                                      Grosvenor Crescent, Belgravia                   1   
                    All Saints Church, Portobello     Devonshire Terrace, Bayswater                   1   
                    Ashley Place, Victoria            Ashley Place, Victoria                          4   
...                                                                                                 ...   
2016-02-06 23:00:00 Whiston Road, Haggerston          New Inn Yard, Shoreditch                        1   
                    Winsland Street, Paddington       Warren Street Station, Euston                   1   
                    Woodstock Street, Mayfair         Imperial Wharf Station                          1   
                    York Hall, Bethnal Green          Shoreditch Park, Hoxton                   

In [12]:
new_result['No of trips']=new_result['Duration']
new_result.drop(columns=['Rental Id'],inplace=True)
new_result.drop(columns=['Duration'],inplace=True)
new_result.drop(columns=['Bike Id'],inplace=True)
new_result.drop(columns=['End Date'],inplace=True)
new_result.drop(columns=['EndStation Id'],inplace=True)
new_result.drop(columns=['StartStation Id'],inplace=True)
new_result

No of trips
Start Date          StartStation Name                 EndStation Name                                      
2016-01-10 00:00:00 Abyssinia Close, Clapham Junction West Kensington Station, West Kensington            1
                    Alderney Street, Pimlico          Chelsea Bridge, Pimlico                             1
                                                      Grosvenor Crescent, Belgravia                       1
                    All Saints Church, Portobello     Devonshire Terrace, Bayswater                       1
                    Ashley Place, Victoria            Ashley Place, Victoria                              4
...                                                                                                     ...
2016-02-06 23:00:00 Whiston Road, Haggerston          New Inn Yard, Shoreditch                            1
                    Winsland Street, Paddington       Warren Street Station, Euston                       1
                    Woodstock Street, Mayfair         Imperial Wharf Station                              1
                    York Hall, Bethnal Green          Shoreditch Park, Hoxton                             1
                                                      Tanner Street, Bermondsey                           1

[517926 rows x 1 columns]